In [1]:
import pandas as pd
import json
import requests
from time import sleep

# Get Data from API using query - 2019

In [2]:
data = []

#query = "q=&deepsearch=on&declaration_year=2019&doc_type=Після+звільнення"
query = "q=Народний+депутат+України&declaration_year=2019&doc_type=Щорічна"


print("Fetching page #%s" % 1)

r = requests.get("http://declarations.com.ua/search?"+ query + "&format=opendata").json()
data += r["results"]["object_list"]

Fetching page #1


In [3]:
for page in range(2, r["results"]["paginator"]["num_pages"] + 1):
    sleep(0.5)
    print("Fetching page #%s" % page)

    subr = requests.get(
        "http://declarations.com.ua/search?"+ query + "&format=opendata&page=%s" % page).json()
    data += subr["results"]["object_list"]

print("Declarations exported %s" % len(data))

with open("NarDep2019.json", "w") as fp:
    json.dump(data, fp)

Fetching page #2
Fetching page #3
Fetching page #4
Fetching page #5
Fetching page #6
Fetching page #7
Fetching page #8
Fetching page #9
Fetching page #10
Fetching page #11
Fetching page #12
Fetching page #13
Fetching page #14
Fetching page #15
Fetching page #16
Fetching page #17
Fetching page #18
Fetching page #19
Fetching page #20
Fetching page #21
Fetching page #22
Fetching page #23
Fetching page #24
Fetching page #25
Fetching page #26
Fetching page #27
Declarations exported 787


In [4]:
print(len(data))

787


In [5]:
declaration = data[0] #json.loads()
declaration.keys()

dict_keys(['guid', 'infocard', 'raw_source', 'unified_source', 'related_entities'])

In [7]:
import pprint
pprint.pprint(declaration)

{'guid': 'nacp_08c243a2-35fe-427d-9b1a-4ce9badaf3ff',
 'infocard': {'created_date': '2020-01-28T12:40:14',
              'declaration_year': 2019,
              'document_type': 'Щорічна',
              'first_name': 'Олександр',
              'id': 'nacp_08c243a2-35fe-427d-9b1a-4ce9badaf3ff',
              'is_corrected': False,
              'last_name': 'Сухов',
              'office': 'народний депутат України ',
              'patronymic': 'Сергійович',
              'position': 'народний депутат України ',
              'source': 'NACP',
              'url': 'https://declarations.com.ua/declaration/nacp_08c243a2-35fe-427d-9b1a-4ce9badaf3ff',
              'user_declarant_id': 827794},
 'raw_source': {'url': 'https://public-api.nazk.gov.ua/v2/documents/08c243a2-35fe-427d-9b1a-4ce9badaf3ff'},
 'related_entities': {'companies': {'all': ['20064120'],
                                    'owned': [],
                                    'related': ['20064120']},
                      'd

# Process Data

In [28]:
declarations = []
not_2019 = 0
corrected = 0
not_NACP = 0
not_yearly = 0



for i, declaration in enumerate(data):
    if (not declaration['related_entities']['documents']['corrected']):
        if declaration['infocard']['source'] == 'NACP':
            if (declaration['infocard']['document_type'] == "Щорічна"):
                if (declaration['unified_source']['step_0']['declarationYear1'] == "2019"):
                    declarations.append(declaration)
                else: not_2019 += 1
            else: not_yearly += 1
        else: not_NACP += 1
    else: corrected +=1
        

print("Not 2019: ", not_2019, 
      "\nNot Yearly: ", not_yearly, 
      "\nNot NACP: ", not_NACP, 
      "\nCorrected: ", corrected)

print("\nTotal declarations: ", len(data),
      "\nRemained declarations: ", len(declarations))

Not 2019:  0 
Not Yearly:  0 
Not NACP:  0 
Corrected:  23

Total declarations:  787 
Remained declarations:  764


**Розділи декларацій, які нас цікавлять:**
  - 7.Цінні папери
  - 8.Корпоративні права
  - 9.Юридичні особи, трасти або інші подібні правові утворення, кінцевим бенефіціарним власником (контролером) яких є суб’єкт декларування або члени його сім’ї
  
  - 15.Робота за сумісництвом суб’єкта декларування
  - 16.Членство суб’єкта декларування в організаціях та їх органах

In [23]:
# Check how many of the declarations have the entities we are looking for

step_7_securities = 0 
step_8_corp_rights = 0 
step_9_owner_beneficiary = 0 
step_15_part_time_work = 0 
step_16_orgs = 0


for index, r in enumerate(declarations):
    if ("step_7" in r['unified_source'].keys()):
        step_7_securities += 1
    if ("step_8" in r['unified_source'].keys()):
        step_8_corp_rights += 1
    if ("step_9" in r['unified_source'].keys()):
        step_9_owner_beneficiary += 1
    if ("step_15" in r['unified_source'].keys()):
        step_15_part_time_work += 1
    if ("step_16" in r['unified_source'].keys()):
        step_16_orgs += 1
        
print(
    "7.Цінні папери: ", step_7_securities, 
    "\n8.Корпоративні права: ", step_8_corp_rights,
    "\n9.Кінцевий бенефіціарний власник (контролер): ", step_9_owner_beneficiary,
    "\n15.Робота за сумісництвом суб’єкта декларування: ", step_15_part_time_work,
    "\n16.Членство суб’єкта декларування в організаціях та їх органах: ", step_16_orgs
)

7.Цінні папери:  82 
8.Корпоративні права:  247 
9.Кінцевий бенефіціарний власник (контролер):  223 
15.Робота за сумісництвом суб’єкта декларування:  144 
16.Членство суб’єкта декларування в організаціях та їх органах:  338


## Get related companies and Orgs

In [64]:
result_list_7 = []
result_list_8 = []
result_list_9 = []
result_list_15 = []
result_list_16 = []

for index, d in enumerate(declarations):
    
    # some general info about declaration
    current_d_id = d['infocard']['id'], 
    current_d_document_type = d['infocard']['document_type'], 
    current_d_year = int(d['unified_source']['step_0']['declarationYear1']),
    current_d_full_name = d['infocard']['last_name'] + ' ' + d['infocard']['first_name'] + ' ' + d['infocard']['patronymic'],
    current_d_office = d['infocard']['office'], 
    current_d_position = d['infocard']['position']
    
    # step 7 -- Цінні папери
    if ("step_7" in d['unified_source'].keys()):
        if len(d['unified_source']['step_7']) > 0: # check if there is data in current step
            for c in d['unified_source']['step_7'].keys():
                company_ua = 0
                company_id = ''
                company_name = ''
                person = ''
                persons= ''

                if ('emitent_ua_company_code' in d['unified_source']['step_7'][c].keys()):
                    if (d['unified_source']['step_7'][c]['emitent_ua_company_code'] != ''):
                            company_ua = 1
                            company_id = d['unified_source']['step_7'][c]['emitent_ua_company_code']
                            company_name = d['unified_source']['step_7'][c]['emitent_ua_company_name']
                if company_id == '':
                    if ('emitent_eng_company_code' in d['unified_source']['step_7'][c].keys()):
                        if (d['unified_source']['step_7'][c]['emitent_eng_company_code'] != ''):
                            company_ua = 0
                            company_id = d['unified_source']['step_7'][c]['emitent_eng_company_code']
                            company_name = d['unified_source']['step_7'][c]['emitent_eng_company_name']

                if ('person' in d['unified_source']['step_7'][c].keys()):
                    person = d['unified_source']['step_7'][c]['person']

                if ('persons' in d['unified_source']['step_7'][c].keys()):
                    persons = d['unified_source']['step_7'][c]['persons']

                result_list_7.append(pd.DataFrame([{'id': current_d_id,
                                                    'index': index,
                                                    'document_type': current_d_document_type,
                                                    'year': current_d_year,
                                                    'full_name': current_d_full_name,
                                                    'office': current_d_office,
                                                    'position': current_d_position,
                                                    'relation_type': 7, 
                                                    'person': person,      
                                                    'persons': persons,
                                                    'company_ua': company_ua,
                                                    'company_id': company_id, 
                                                    'company_name': company_name}]))
                
                
    # step 8 -- Корпоративні права
    if ("step_8" in d['unified_source'].keys()):
        if len(d['unified_source']['step_8']) > 0: # check if there is data in current step
            for c in d['unified_source']['step_8'].keys():
                company_ua = 0
                company_id = ''
                company_name = ''
                person = ''
                persons= ''

                if d['unified_source']['step_8'][c]['country'] == '1':
                    company_ua = 1

                if ('corporate_rights_company_code' in d['unified_source']['step_8'][c].keys()):
                    company_id = d['unified_source']['step_8'][c]['corporate_rights_company_code']

                if ('name' in d['unified_source']['step_8'][c].keys()):
                    if (d['unified_source']['step_8'][c]['name'] != ''):
                        company_name = d['unified_source']['step_8'][c]['name']
                if company_name == '':
                    if ('en_name' in d['unified_source']['step_8'][c].keys()):
                        if (d['unified_source']['step_8'][c]['en_name'] != ''):
                            company_name = d['unified_source']['step_8'][c]['en_name']

                if ('person' in d['unified_source']['step_8'][c].keys()):
                    person = d['unified_source']['step_8'][c]['person']

                if ('persons' in d['unified_source']['step_8'][c].keys()):
                    persons = d['unified_source']['step_8'][c]['persons']

                result_list_8.append(pd.DataFrame([{'id': current_d_id,
                                                    'index': index,
                                                    'document_type': current_d_document_type,
                                                    'year': current_d_year,
                                                    'full_name': current_d_full_name,
                                                    'office': current_d_office,
                                                    'position': current_d_position,
                                                    'relation_type': 8, 
                                                    'person': person,      
                                                    'persons': persons,
                                                    'company_ua': company_ua,
                                                    'company_id': company_id, 
                                                    'company_name': company_name}]))
            
            
    
    # step 9 -- Кінцевий бенефіціарний власник (контролер)
    if ("step_9" in d['unified_source'].keys()):
        if len(d['unified_source']['step_9']) > 0: # check if there is data in current step
            for c in d['unified_source']['step_9'].keys():
                company_ua = 0
                company_id = ''
                company_name = ''
                person = ''
                persons= ''

                try:
                    if d['unified_source']['step_9'][c]['country_beneficial_owner'] == '1':
                        company_ua = 1
                except KeyError:
                    if d['unified_source']['step_9'][c]['country'] == '1':
                        company_ua = 1

                if ('beneficial_owner_company_code' in d['unified_source']['step_9'][c].keys()):
                    company_id = d['unified_source']['step_9'][c]['beneficial_owner_company_code']

                if ('name' in d['unified_source']['step_9'][c].keys()):
                    if (d['unified_source']['step_9'][c]['name'] != ''):
                        company_name = d['unified_source']['step_9'][c]['name']
                if company_name == '':
                    if ('en_name' in d['unified_source']['step_9'][c].keys()):
                        if (d['unified_source']['step_9'][c]['en_name'] != ''):
                            company_name = d['unified_source']['step_9'][c]['en_name']

                if ('person' in d['unified_source']['step_9'][c].keys()):
                    person = d['unified_source']['step_9'][c]['person']

                if ('persons' in d['unified_source']['step_9'][c].keys()):
                    persons = d['unified_source']['step_9'][c]['persons']


                result_list_9.append(pd.DataFrame([{'id': current_d_id,
                                                    'index': index,
                                                    'document_type': current_d_document_type,
                                                    'year': current_d_year,
                                                    'full_name': current_d_full_name,
                                                    'office': current_d_office,
                                                    'position': current_d_position,
                                                    'relation_type': 9, 
                                                    'person': person,      
                                                    'persons': persons,
                                                    'company_ua': company_ua,
                                                    'company_id': company_id, 
                                                    'company_name': company_name}]))
            
        
    # step 15 -- Робота за сумісництвом суб’єкта декларування
    if ("step_15" in d['unified_source'].keys()):
        if len(d['unified_source']['step_15']) > 0: # check if there is data in current step
            for c in d['unified_source']['step_15'].keys():
                company_ua = 0
                company_id = ''
                company_name = ''
                person = ''
                persons = ''
                description = ''
                emitent_citizen = ''
                paid = ''

                if ('emitent_ua_company_code' in d['unified_source']['step_15'][c].keys()):
                    if (d['unified_source']['step_15'][c]['emitent_ua_company_code'] != ''):
                            company_ua = 1
                            company_id = d['unified_source']['step_15'][c]['emitent_ua_company_code']
                            company_name = d['unified_source']['step_15'][c]['emitent_ua_company_name']
                if company_id == '':
                    if ('emitent_eng_company_code' in d['unified_source']['step_15'][c].keys()):
                        if (d['unified_source']['step_15'][c]['emitent_eng_company_code'] != ''):
                            company_ua = 0
                            company_id = d['unified_source']['step_15'][c]['emitent_eng_company_code']
                            company_name = d['unified_source']['step_15'][c]['emitent_eng_company_name']


                if ('description' in d['unified_source']['step_15'][c].keys()):
                    description = d['unified_source']['step_15'][c]['description']

                if ('emitent_citizen' in d['unified_source']['step_15'][c].keys()):
                    emitent_citizen = d['unified_source']['step_15'][c]['emitent_citizen']

                if ('paid' in d['unified_source']['step_15'][c].keys()):
                    paid = d['unified_source']['step_15'][c]['paid']

                result_list_15.append(pd.DataFrame([{'id': current_d_id,
                                                    'index': index,
                                                    'document_type': current_d_document_type,
                                                    'year': current_d_year,
                                                    'full_name': current_d_full_name,
                                                    'office': current_d_office,
                                                    'position': current_d_position,
                                                    'relation_type': 15, 
                                                    'person': person,      
                                                    'persons': persons,
                                                    'company_ua': company_ua,
                                                    'company_id': company_id, 
                                                    'company_name': company_name,
                                                    'description': description, 
                                                    'emitent_citizen': emitent_citizen,
                                                    'paid': paid}]))

            
    # step 16 -- Членство суб’єкта декларування в організаціях та їх органах
    if ("step_16" in d['unified_source'].keys()):
        # for orgs
        if len(d['unified_source']['step_16']['org']) > 0: # check if there is data in current step
            for c in d['unified_source']['step_16']['org'].keys():
                org_name = ''
                org_type = ''
                org_id = ''
                org_unit_name = 'org'
                org_unit_type = 'org'

                if ('objectName' in d['unified_source']['step_16']['org'][c].keys()):
                    org_name = d['unified_source']['step_16']['org'][c]['objectName']

                if ('objectType' in d['unified_source']['step_16']['org'][c].keys()):
                    org_type = d['unified_source']['step_16']['org'][c]['objectType']

                if ('reestrCode' in d['unified_source']['step_16']['org'][c].keys()):
                    org_id = d['unified_source']['step_16']['org'][c]['reestrCode']

                result_list_16.append(pd.DataFrame([{'id': current_d_id,
                                                    'index': index,
                                                    'document_type': current_d_document_type,
                                                    'year': current_d_year,
                                                    'full_name': current_d_full_name,
                                                    'office': current_d_office,
                                                    'position': current_d_position,
                                                    'relation_type': 16, 
                                                    'org_name': org_name,
                                                    'org_type': org_type,
                                                    'org_id': org_id,
                                                    'org_unit_name': org_unit_name,
                                                    'org_unit_type': org_unit_type}]))
            
        # for org parts
        if len(d['unified_source']['step_16']['part_org']) > 0: # check if there is data in current step
            for c in d['unified_source']['step_16']['part_org'].keys():
                org_name = ''
                org_type = ''
                org_id = ''
                org_unit_name = ''
                org_unit_type = ''

                if ('objectName' in d['unified_source']['step_16']['part_org'][c].keys()):
                    org_name = d['unified_source']['step_16']['part_org'][c]['objectName']

                if ('objectType' in d['unified_source']['step_16']['part_org'][c].keys()):
                    org_type = d['unified_source']['step_16']['part_org'][c]['objectType']

                if ('reestrCode' in d['unified_source']['step_16']['part_org'][c].keys()):
                    org_id = d['unified_source']['step_16']['part_org'][c]['reestrCode']  

                if ('unitName' in d['unified_source']['step_16']['part_org'][c].keys()):
                    org_unit_name = d['unified_source']['step_16']['part_org'][c]['unitName']   

                if ('unitType' in d['unified_source']['step_16']['part_org'][c].keys()):
                    org_unit_type = d['unified_source']['step_16']['part_org'][c]['unitType']

                result_list_16.append(pd.DataFrame([{'id': current_d_id,
                                                    'index': index,
                                                    'document_type': current_d_document_type,
                                                    'year': current_d_year,
                                                    'full_name': current_d_full_name,
                                                    'office': current_d_office,
                                                    'position': current_d_position,
                                                    'relation_type': 16, 
                                                    'org_name': org_name,
                                                    'org_type': org_type,
                                                    'org_id': org_id,
                                                    'org_unit_name': org_unit_name,
                                                    'org_unit_type': org_unit_type}]))

In [65]:
print(len(result_list_7),
      len(result_list_8),
      len(result_list_9),
      len(result_list_15),
      len(result_list_16),
)

172 917 2038 298 990


Create summary dataframe for all companies (from steps 7,8,9,15)

In [78]:
companies_df = pd.DataFrame({
    'id': [], 
    'index': [], 
    'document_type': [], 
    'year': [], 
    'full_name': [], 
    'office': [], 
    'position': [],
    'relation_type': [], 
    'person': [], 
    'persons': [], 
    'company_ua': [],
    'company_id': [], 
    'company_name': [],
    'description': [],
    'emitent_citizen': [],
    'paid': []
})                           

In [76]:
companies_df = companies_df.append(result_list_7)
companies_df = companies_df.append(result_list_8)
companies_df = companies_df.append(result_list_9)
companies_df = companies_df.append(result_list_15)

In [77]:
companies_df

,id,index,document_type,year,full_name,office,position,relation_type,person,persons,company_ua,company_id,company_name,description,emitent_citizen,paid
0,"(nacp_b352acb8-68bc-44b4-a3a4-eebde0e00953,)",22.0,"(Щорічна,)","(2019,)","(Кучер Микола Іванович,)","(Верховна Рада України,)",народний депутат України,7.0,,j,0.0,1870404,Tansy Holdings Ltd.,NaN,NaN,NaN
0,"(nacp_b352acb8-68bc-44b4-a3a4-eebde0e00953,)",22.0,"(Щорічна,)","(2019,)","(Кучер Микола Іванович,)","(Верховна Рада України,)",народний депутат України,7.0,,j,0.0,C71563,Azalea Holdings Ltd.,NaN,NaN,NaN
0,"(nacp_6003cb68-4147-47f0-8d34-8045fd596c88,)",31.0,"(Щорічна,)","(2019,)","(Драбовський Анатолій Григорович,)","(Верховна Рада України,)",Народний депутат України,7.0,,,1.0,00373758,"ВАТ ""Вінницький олійно-жировий комбінат""",NaN,NaN,NaN
0,"(nacp_f041a3d4-0702-41cf-97c4-0b7ebbc8bc0a,)",32.0,"(Щорічна,)","(2019,)","(Третьякова Галина Миколаївна,)","(Верховна Рада України,)",Народний депутат України,7.0,,,1.0,34486135,"ТОВ ""КУА""ІНВЕСТИЦІЙНИЙ КАПІТАЛ УКРАЇНА""",NaN,NaN,NaN
0,"(nacp_b51cf2a4-4b0b-411f-a5a8-2399ba6a7c6d,)",41.0,"(Щорічна,)","(2019,)","(Приходько Борис Вікторович,)","(Верховна Рада України,)",Народний депутат України,7.0,,,1.0,00013480,Міністерство фінансів України,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,"(nacp_81e98180-e237-4347-ac8f-e39ab8be132a,)",735.0,"(Щорічна,)","(2019,)","(Вагнєр Вікторія Олександрівна,)","(Верховна Рада України,)","Член комітету з питань здоров я нації, медично...",15.0,,,1.0,02012022,"ДУ ""Інститут педіатрії, акушерства і гінеколог...",молодший науковий спеціаліст,"Юридична особа, зареєстрована в Україні",Оплачувана
0,"(nacp_0d4ff3f0-2254-400b-9abf-58c497354ee0,)",736.0,"(Щорічна,)","(2019,)","(Колесников Вячеслав Ігорович,)",(Апарат Верховної Ради України/Пологівська міс...,Помічник-консультант народного депутата Україн...,15.0,,,1.0,02070849,"НАЦІОНАЛЬНИЙ УНІВЕРСИТЕТ ""ЗАПОРІЗЬКА ПОЛІТЕХНІ...",Доцент кафедри економіки та митноЇ справи,"Юридична особа, зареєстрована в Україні",Оплачувана
0,"(nacp_0d4ff3f0-2254-400b-9abf-58c497354ee0,)",736.0,"(Щорічна,)","(2019,)","(Колесников Вячеслав Ігорович,)",(Апарат Верховної Ради України/Пологівська міс...,Помічник-консультант народного депутата Україн...,15.0,,,1.0,02070849,"НАЦІОНАЛЬНИЙ УНІВЕРСИТЕТ ""ЗАПОРІЗЬКА ПОЛІТЕХНІКА""",Старший викладач кафедри управління персоналом...,"Юридична особа, зареєстрована в Україні",Оплачувана
0,"(nacp_19e776f3-8df7-4e12-90b3-6df280179ebe,)",749.0,"(Щорічна,)","(2019,)","(Бакумов Олександр Сергійович,)","(Верховна Рада України,)","Народний депутат України, Голова підкомітету з...",15.0,,,1.0,08571096,Харківський національний університет внутрішні...,Доцент,"Юридична особа, зареєстрована в Україні",Оплачувана


Create summary dataframe for all organizations (from step 16)

In [80]:
organizations_df = pd.DataFrame({
    'id': [],
    'index': [], 
    'document_type': [], 
    'year': [], 
    'full_name': [], 
    'office': [], 
    'position': [],
    'relation_type': [], 
    'org_name': [], 
    'org_type': [], 
    'org_id': [],
    'org_unit_name': [], 
    'org_unit_type': []
})   

In [81]:
organizations_df = organizations_df.append(result_list_16)
organizations_df

,id,index,document_type,year,full_name,office,position,relation_type,org_name,org_type,org_id,org_unit_name,org_unit_type
0,"(nacp_3a9e9b55-3621-4436-9fd6-319c861a4624,)",1.0,"(Щорічна,)","(2019,)","(Вельможний Сергій Анатолійович,)","(народний депутат України,)",народний депутат України,16.0,Національна асоціація адвокатів України,Саморегулівне чи самоврядне професійне об’єднання,38488439,org,org
0,"(nacp_da197e09-3ff3-4ce6-8e12-3e813fa3e823,)",15.0,"(Щорічна,)","(2019,)","(Лунченко Валерій Валерійович,)","(Верховна Рада України,)",Народний депутат України,16.0,Благодійна організація благодійний фонд Валері...,Благодійна організація,38629273,org,org
0,"(nacp_da197e09-3ff3-4ce6-8e12-3e813fa3e823,)",15.0,"(Щорічна,)","(2019,)","(Лунченко Валерій Валерійович,)","(Верховна Рада України,)",Народний депутат України,16.0,"Громадська організація ""Фундація розвитку зака...",Громадське об’єднання,40896515,org,org
0,"(nacp_da197e09-3ff3-4ce6-8e12-3e813fa3e823,)",15.0,"(Щорічна,)","(2019,)","(Лунченко Валерій Валерійович,)","(Верховна Рада України,)",Народний депутат України,16.0,"ГРОМАДСЬКА ОРГАНІЗАЦІЯ ""ВСЕУКРАЇНСЬКА АСОЦІАЦІ...",Громадське об’єднання,41092302,org,org
0,"(nacp_da197e09-3ff3-4ce6-8e12-3e813fa3e823,)",15.0,"(Щорічна,)","(2019,)","(Лунченко Валерій Валерійович,)","(Верховна Рада України,)",Народний депутат України,16.0,Благодійна організація благодійний фонд Валері...,Благодійна організація,38629273,Засновник,Керівні органи
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,"(nacp_5ded6455-0200-46ea-b365-ea7aa392b170,)",756.0,"(Щорічна,)","(2019,)","(Луцков Володимир Ігорович,)","(Апарат Верховної ради України, Товариство з о...",Помічник консультант на громадських засадах на...,16.0,Громадська організація ПІСКІВКА СПОРТЦЕНТР,Громадське об’єднання,42113019,Голова правління,Керівні органи
0,"(nacp_12a6d842-3258-444a-953e-cc576f726b7a,)",757.0,"(Щорічна,)","(2019,)","(Луцков Володимир Ігорович,)","(Апарат Верховної ради України, Товариство з о...",Помічник консультант на громадських засадах на...,16.0,Громадська організація ПІСКІВКА СПОРТЦЕНТР,Громадське об’єднання,42113019,org,org
0,"(nacp_12a6d842-3258-444a-953e-cc576f726b7a,)",757.0,"(Щорічна,)","(2019,)","(Луцков Володимир Ігорович,)","(Апарат Верховної ради України, Товариство з о...",Помічник консультант на громадських засадах на...,16.0,Громадська організація ПІСКІВКА СПОРТЦЕНТР,Громадське об’єднання,42113019,Голова правління,Керівні органи
0,"(nacp_e9105bf5-1457-438d-8c63-4626c6dab6c5,)",758.0,"(Щорічна,)","(2019,)","(Тимченко Тетяна Юріївна,)","(Міністерство юстиції України, Департамент дер...",начальник відділу розгляду запитів і звернень ...,16.0,Первинна профспілкова організація Міністерства...,Громадське об’єднання,42171490,org,org
